Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.900489
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.314119
Minibatch accuracy: 62.5%
Validation accuracy: 42.1%
Minibatch loss at step 100: 1.843671
Minibatch accuracy: 75.0%
Validation accuracy: 68.2%
Minibatch loss at step 150: 1.073419
Minibatch accuracy: 75.0%
Validation accuracy: 73.7%
Minibatch loss at step 200: 1.924803
Minibatch accuracy: 37.5%
Validation accuracy: 78.0%
Minibatch loss at step 250: 0.571881
Minibatch accuracy: 87.5%
Validation accuracy: 75.8%
Minibatch loss at step 300: 0.724763
Minibatch accuracy: 87.5%
Validation accuracy: 79.2%
Minibatch loss at step 350: 1.398892
Minibatch accuracy: 50.0%
Validation accuracy: 78.9%
Minibatch loss at step 400: 1.100714
Minibatch accuracy: 68.8%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.442779
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Minibatch loss at step 500: 0.767835
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [26]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    #ANKIT NOTE - Batch, Height, Width, Channel strides in the row matrix below
    
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    #ANKITNOTE - Still doing convolution but with strides of 1
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [28]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.158651
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.513768
Minibatch accuracy: 43.8%
Validation accuracy: 37.5%
Minibatch loss at step 100: 1.733822
Minibatch accuracy: 50.0%
Validation accuracy: 64.6%
Minibatch loss at step 150: 1.093665
Minibatch accuracy: 75.0%
Validation accuracy: 73.8%
Minibatch loss at step 200: 1.732018
Minibatch accuracy: 43.8%
Validation accuracy: 73.9%
Minibatch loss at step 250: 0.647450
Minibatch accuracy: 93.8%
Validation accuracy: 75.7%
Minibatch loss at step 300: 0.776790
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 350: 1.293322
Minibatch accuracy: 56.2%
Validation accuracy: 77.7%
Minibatch loss at step 400: 1.223022
Minibatch accuracy: 56.2%
Validation accuracy: 81.5%
Minibatch loss at step 450: 0.447033
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Minibatch loss at step 500: 0.661239
Minibatch accuracy: 68.8%
Validation accuracy: 82.0%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [29]:
#Trying only dropout regualization

In [35]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    drop1 = tf.nn.dropout(hidden, 0.7)
    pool = tf.nn.max_pool(drop1, [1, 2, 2, 1], [1, 1, 1, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    drop2 = tf.nn.dropout(hidden, 0.7)
    pool = tf.nn.max_pool(drop2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    drop3 = tf.nn.dropout(hidden, 0.7)
    return tf.matmul(drop3, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [36]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.199826
Minibatch accuracy: 6.2%
Validation accuracy: 10.5%
Minibatch loss at step 50: 1.182551
Minibatch accuracy: 56.2%
Validation accuracy: 50.5%
Minibatch loss at step 100: 1.631202
Minibatch accuracy: 62.5%
Validation accuracy: 64.1%
Minibatch loss at step 150: 1.055494
Minibatch accuracy: 68.8%
Validation accuracy: 68.2%
Minibatch loss at step 200: 1.656828
Minibatch accuracy: 50.0%
Validation accuracy: 72.7%
Minibatch loss at step 250: 0.657162
Minibatch accuracy: 81.2%
Validation accuracy: 72.1%
Minibatch loss at step 300: 1.007711
Minibatch accuracy: 81.2%
Validation accuracy: 73.1%
Minibatch loss at step 350: 1.562309
Minibatch accuracy: 56.2%
Validation accuracy: 74.2%
Minibatch loss at step 400: 1.206878
Minibatch accuracy: 68.8%
Validation accuracy: 74.5%
Minibatch loss at step 450: 0.905786
Minibatch accuracy: 75.0%
Validation accuracy: 77.4%
Minibatch loss at step 500: 0.820724
Minibatch accuracy: 68.8%
Validation accuracy: 77.2%
Mi

Validation accuracy: 84.8%
Minibatch loss at step 4550: 0.444401
Minibatch accuracy: 81.2%
Validation accuracy: 85.2%
Minibatch loss at step 4600: 0.014679
Minibatch accuracy: 100.0%
Validation accuracy: 84.8%
Minibatch loss at step 4650: 0.402283
Minibatch accuracy: 93.8%
Validation accuracy: 85.4%
Minibatch loss at step 4700: 0.397949
Minibatch accuracy: 93.8%
Validation accuracy: 84.9%
Minibatch loss at step 4750: 0.614722
Minibatch accuracy: 81.2%
Validation accuracy: 85.5%
Minibatch loss at step 4800: 0.867795
Minibatch accuracy: 87.5%
Validation accuracy: 85.0%
Minibatch loss at step 4850: 0.290323
Minibatch accuracy: 93.8%
Validation accuracy: 85.2%
Minibatch loss at step 4900: 0.397898
Minibatch accuracy: 93.8%
Validation accuracy: 84.9%
Minibatch loss at step 4950: 0.273994
Minibatch accuracy: 93.8%
Validation accuracy: 85.6%
Minibatch loss at step 5000: 0.351724
Minibatch accuracy: 87.5%
Validation accuracy: 85.2%
Test accuracy: 90.9%


In [51]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 128

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.1, global_step, 10000, 0.95, staircase=True)
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    drop1 = tf.nn.dropout(hidden, 0.7)
    pool = tf.nn.max_pool(drop1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    drop2 = tf.nn.dropout(hidden, 0.7)
    pool = tf.nn.max_pool(drop2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    drop3 = tf.nn.dropout(hidden, 0.7)
    return tf.matmul(drop3, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [55]:
num_steps = 50001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.961430
Minibatch accuracy: 6.2%
Validation accuracy: 11.8%
Minibatch loss at step 50: 1.178514
Minibatch accuracy: 62.5%
Validation accuracy: 64.3%
Minibatch loss at step 100: 1.536515
Minibatch accuracy: 43.8%
Validation accuracy: 71.9%
Minibatch loss at step 150: 0.939365
Minibatch accuracy: 68.8%
Validation accuracy: 74.7%
Minibatch loss at step 200: 2.099048
Minibatch accuracy: 37.5%
Validation accuracy: 77.0%
Minibatch loss at step 250: 0.765158
Minibatch accuracy: 68.8%
Validation accuracy: 72.5%
Minibatch loss at step 300: 0.570178
Minibatch accuracy: 87.5%
Validation accuracy: 71.0%
Minibatch loss at step 350: 1.326662
Minibatch accuracy: 62.5%
Validation accuracy: 78.8%
Minibatch loss at step 400: 0.970527
Minibatch accuracy: 62.5%
Validation accuracy: 76.5%
Minibatch loss at step 450: 0.564982
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 500: 0.870990
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Mi

Validation accuracy: 85.4%
Minibatch loss at step 4600: 0.145114
Minibatch accuracy: 93.8%
Validation accuracy: 86.3%
Minibatch loss at step 4650: 0.268115
Minibatch accuracy: 93.8%
Validation accuracy: 85.5%
Minibatch loss at step 4700: 0.444377
Minibatch accuracy: 87.5%
Validation accuracy: 84.8%
Minibatch loss at step 4750: 0.591744
Minibatch accuracy: 81.2%
Validation accuracy: 85.4%
Minibatch loss at step 4800: 0.319383
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 4850: 0.297314
Minibatch accuracy: 81.2%
Validation accuracy: 85.9%
Minibatch loss at step 4900: 0.230310
Minibatch accuracy: 93.8%
Validation accuracy: 86.3%
Minibatch loss at step 4950: 0.235049
Minibatch accuracy: 93.8%
Validation accuracy: 86.1%
Minibatch loss at step 5000: 0.355304
Minibatch accuracy: 93.8%
Validation accuracy: 85.2%
Minibatch loss at step 5050: 0.411655
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 5100: 0.307597
Minibatch accuracy: 87.5%

Validation accuracy: 87.1%
Minibatch loss at step 9100: 0.303158
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Minibatch loss at step 9150: 0.548981
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Minibatch loss at step 9200: 0.101319
Minibatch accuracy: 100.0%
Validation accuracy: 86.6%
Minibatch loss at step 9250: 0.657276
Minibatch accuracy: 75.0%
Validation accuracy: 85.8%
Minibatch loss at step 9300: 0.559556
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 9350: 0.310539
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 9400: 0.505736
Minibatch accuracy: 81.2%
Validation accuracy: 87.2%
Minibatch loss at step 9450: 0.241447
Minibatch accuracy: 93.8%
Validation accuracy: 86.4%
Minibatch loss at step 9500: 0.790648
Minibatch accuracy: 87.5%
Validation accuracy: 86.1%
Minibatch loss at step 9550: 0.546798
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 9600: 0.372330
Minibatch accuracy: 87.5

Minibatch loss at step 13550: 0.243406
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 13600: 0.565980
Minibatch accuracy: 87.5%
Validation accuracy: 87.6%
Minibatch loss at step 13650: 0.233814
Minibatch accuracy: 87.5%
Validation accuracy: 87.9%
Minibatch loss at step 13700: 0.269740
Minibatch accuracy: 93.8%
Validation accuracy: 87.4%
Minibatch loss at step 13750: 0.002448
Minibatch accuracy: 100.0%
Validation accuracy: 86.7%
Minibatch loss at step 13800: 0.305880
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 13850: 0.243220
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 13900: 0.418978
Minibatch accuracy: 87.5%
Validation accuracy: 88.0%
Minibatch loss at step 13950: 0.276573
Minibatch accuracy: 93.8%
Validation accuracy: 87.2%
Minibatch loss at step 14000: 0.621512
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 14050: 0.513424
Minibatch accuracy: 75.0%
Validation acc

Minibatch loss at step 18050: 0.510401
Minibatch accuracy: 75.0%
Validation accuracy: 87.7%
Minibatch loss at step 18100: 0.765422
Minibatch accuracy: 75.0%
Validation accuracy: 87.7%
Minibatch loss at step 18150: 0.321922
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
Minibatch loss at step 18200: 0.138057
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%
Minibatch loss at step 18250: 0.511082
Minibatch accuracy: 75.0%
Validation accuracy: 87.0%
Minibatch loss at step 18300: 0.249771
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 18350: 0.987287
Minibatch accuracy: 68.8%
Validation accuracy: 87.9%
Minibatch loss at step 18400: 0.226094
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 18450: 0.332756
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 18500: 0.081232
Minibatch accuracy: 100.0%
Validation accuracy: 87.4%
Minibatch loss at step 18550: 0.560991
Minibatch accuracy: 75.0%
Validation acc

...

Minibatch loss at step 31450: 0.334454
Minibatch accuracy: 87.5%
Validation accuracy: 88.9%
Minibatch loss at step 31500: 0.700370
Minibatch accuracy: 68.8%
Validation accuracy: 87.8%
Minibatch loss at step 31550: 0.776463
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 31600: 1.033062
Minibatch accuracy: 81.2%
Validation accuracy: 89.2%
Minibatch loss at step 31650: 0.180406
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%
Minibatch loss at step 31700: 0.187953
Minibatch accuracy: 93.8%
Validation accuracy: 88.9%
Minibatch loss at step 31750: 0.172395
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 31800: 0.206841
Minibatch accuracy: 93.8%
Validation accuracy: 88.6%
Minibatch loss at step 31850: 0.413345
Minibatch accuracy: 93.8%
Validation accuracy: 89.2%
Minibatch loss at step 31900: 0.437175
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 31950: 0.724287
Minibatch accuracy: 81.2%
Validation accu

Validation accuracy: 88.9%
Minibatch loss at step 35950: 0.371857
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 36000: 0.015267
Minibatch accuracy: 100.0%
Validation accuracy: 88.9%
Minibatch loss at step 36050: 0.356528
Minibatch accuracy: 81.2%
Validation accuracy: 88.4%
Minibatch loss at step 36100: 0.307504
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 36150: 0.685727
Minibatch accuracy: 75.0%
Validation accuracy: 89.0%
Minibatch loss at step 36200: 0.156053
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 36250: 0.295800
Minibatch accuracy: 93.8%
Validation accuracy: 89.1%
Minibatch loss at step 36300: 0.222380
Minibatch accuracy: 93.8%
Validation accuracy: 88.6%
Minibatch loss at step 36350: 0.299777
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Minibatch loss at step 36400: 0.228251
Minibatch accuracy: 93.8%
Validation accuracy: 88.3%
Minibatch loss at step 36450: 0.583277
Minibatch acc

Validation accuracy: 88.9%
Minibatch loss at step 40400: 0.232283
Minibatch accuracy: 93.8%
Validation accuracy: 89.0%
Minibatch loss at step 40450: 0.496275
Minibatch accuracy: 87.5%
Validation accuracy: 88.0%
Minibatch loss at step 40500: 0.304760
Minibatch accuracy: 93.8%
Validation accuracy: 89.2%
Minibatch loss at step 40550: 0.604168
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 40600: 0.018854
Minibatch accuracy: 100.0%
Validation accuracy: 89.1%
Minibatch loss at step 40650: 0.587016
Minibatch accuracy: 81.2%
Validation accuracy: 88.5%
Minibatch loss at step 40700: 0.688598
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 40750: 1.010178
Minibatch accuracy: 75.0%
Validation accuracy: 89.3%
Minibatch loss at step 40800: 0.524867
Minibatch accuracy: 81.2%
Validation accuracy: 89.1%
Minibatch loss at step 40850: 0.888437
Minibatch accuracy: 81.2%
Validation accuracy: 89.3%
Minibatch loss at step 40900: 0.067812
Minibatch acc

Minibatch loss at step 44850: 0.230039
Minibatch accuracy: 93.8%
Validation accuracy: 89.0%
Minibatch loss at step 44900: 0.385029
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 44950: 0.162075
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%
Minibatch loss at step 45000: 0.476523
Minibatch accuracy: 81.2%
Validation accuracy: 88.7%
Minibatch loss at step 45050: 0.315227
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 45100: 0.134825
Minibatch accuracy: 100.0%
Validation accuracy: 88.8%
Minibatch loss at step 45150: 0.408178
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 45200: 0.320503
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 45250: 0.654469
Minibatch accuracy: 75.0%
Validation accuracy: 88.6%
Minibatch loss at step 45300: 0.208114
Minibatch accuracy: 93.8%
Validation accuracy: 89.1%
Minibatch loss at step 45350: 0.784489
Minibatch accuracy: 75.0%
Validation acc

Minibatch loss at step 49300: 0.404859
Minibatch accuracy: 81.2%
Validation accuracy: 88.9%
Minibatch loss at step 49350: 0.275730
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 49400: 0.796333
Minibatch accuracy: 81.2%
Validation accuracy: 88.8%
Minibatch loss at step 49450: 0.541663
Minibatch accuracy: 81.2%
Validation accuracy: 88.8%
Minibatch loss at step 49500: 0.149493
Minibatch accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss at step 49550: 0.213959
Minibatch accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss at step 49600: 0.896608
Minibatch accuracy: 62.5%
Validation accuracy: 88.8%
Minibatch loss at step 49650: 0.232202
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 49700: 0.655701
Minibatch accuracy: 81.2%
Validation accuracy: 89.0%
Minibatch loss at step 49750: 0.122986
Minibatch accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss at step 49800: 0.479878
Minibatch accuracy: 87.5%
Validation accu